In [1]:
import pickle
from class_definitions import Annotation, BertContainer
import pandas as pd

In [2]:
#pkl_file = open("Container16_notities_2017_deel1_cleaned.csv---2503.conll_avelli.pkl", "rb")
pkl_filepath = '../sample_data/BERTContainers/Container_notities_2017_deel1_cleaned.csv---25032_avelli.pkl'

pkl_file = open(pkl_filepath, "rb")
data = pickle.load(pkl_file)
print(data)

[<class_definitions.BertContainer object at 0x0000025F6ED4AC18>, <class_definitions.BertContainer object at 0x0000025F71008128>]


In [4]:
# self.key = key
# self.annotator = annotator
# self.sen_id = sen_id
# self.sen = sen
# self.encoding = encoding
# self.annot = []

domain_list = ['.B152: Stemming', 'insp tol']
gradation = ['STM 0', 'STM 1']

sentence_count = 0
sentence_with_label = 0
sentence_with_domain = 0

for BertContainer in data:
    sentence_count += 1
    sentence_with_domain_bool = False
    if len(BertContainer.annot) != 0:
        sentence_with_label += 1
        
    for annotation in BertContainer.annot:
        print(annotation.label)
        if annotation.label in domain_list:
            
            sentence_with_domain_bool = True
    if sentence_with_domain_bool == True:
        sentence_with_domain += 0
sentence_with_label

.B152: Stemming
STM 0
STM 1
stm\_reaction
.B152: Stemming


1

In [34]:
domain_list = ['.B152: Stemming', 'insp tol']
gradation = ['STM 0', 'STM 1', 'INS 4']

labels_count = 0
sentences_count = 0


for BertContainer in data:
    # gather annotations
    annotation_list = []
    if len(BertContainer.annot) != 0:
        for annotation in BertContainer.annot:
            annotation_list.append(annotation.label)
    print(annotation_list)
    
    

[]
['.B152: Stemming', 'STM 0', 'STM 1', 'stm\\_reaction', '.B152: Stemming']
